# Decision tree Classification

## Preperation 

### Importing needed libraries

In [43]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading Data

In [44]:
train_data = pd.read_csv("../../data/220510_train_data_preprocessed.csv", sep=';')
test_data = pd.read_csv("../../data/220510_test_data_preprocessed.csv", sep=";")
train_data.head()

,id,label,tweet,tweet_converted_emojis,n_mentions,hashtags,without_punctuation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,1,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,1,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['user', 'when', 'a', 'father', 'is', 'dysfunc...","['user', 'father', 'dysfunctional', 'selfish',...",['run'],"['user', 'father', 'dysfunct', 'selfish', 'dra...",['run'],"['user', 'father', 'dysfunctional', 'selfish',...",['run']
1,2,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,2,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'user', 'thanks', 'for', 'lyft', 'cre...","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']","['user', 'user', 'thank', 'lyft', 'credit', 'c...","['lyft', 'disapoint', 'getthank']","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']"
2,3,0,bihday your majesty,bihday your majesty,0,[],bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']",[],"['bihday', 'majesti']",[],"['bihday', 'majesty']",[]
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,0,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['model', 'i', 'love', 'u', 'take', 'with', 'u...","['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model']
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation,0,['motivation'],factsguide society now motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']",['motivation'],"['factsguid', 'societi', 'motiv']",['motiv'],"['factsguide', 'society', 'motivation']",['motivation']


In [78]:
train_data.corr()

,id,label,n_mentions
id,1.000000,0.002780,0.000133
label,0.002780,1.000000,0.078515
n_mentions,0.000133,0.078515,1.000000


## Feature Selection Lemmatized

In [45]:
"""features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]
X = data[features]
y = data.label
X.head()"""

'features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]\nX = data[features]\ny = data.label\nX.head()'

## Bag of words model

Create the bag of words from the data

In [46]:
"""import texthero as hero
from texthero import preprocessing
X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=15000))
X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=15000))
X"""

'import texthero as hero\nfrom texthero import preprocessing\nX["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=15000))\nX["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=15000))\nX'

Reduce the dimension of the vector

In [47]:
"""X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))
X"""

'X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))\nX["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))\nX'

Split vector into columns

In [48]:
"""X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
X[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)
X"""

'X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)\nX[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)\nX'

# Lemmatisation

In [49]:
features = ["lemmatized_tokens", "lemmatized_hashtags", "n_mentions"]

X = train_data[features]
y = train_data.label

X.head()

,lemmatized_tokens,lemmatized_hashtags,n_mentions
0,"['user', 'father', 'dysfunctional', 'selfish',...",['run'],1
1,"['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']",2
2,"['bihday', 'majesty']",[],0
3,"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],0
4,"['factsguide', 'society', 'motivation']",['motivation'],0


### Using TF-IDF Vectorizer

In [50]:
X.shape

(31962, 3)

In [51]:
tf = TfidfVectorizer()

X_vec = tf.fit(X["lemmatized_tokens"])
X_lem = X_vec.transform(X["lemmatized_tokens"])

In [52]:
X_lem.shape

(31962, 40632)

## Split data

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_lem, y, test_size=0.2, random_state = 17, stratify=y)

In [54]:
print(f"Shape X_train {X_train.shape}")
print(f"Shape X_test {X_test.shape}")

Shape X_train (25569, 40632)
Shape X_test (6393, 40632)


## Fitting model

In [55]:
result_precision_score = []
result_recall_score = []
result_accuracy_score = []
result_f1_score = []
max_depth = 100

for i in range(5, max_depth):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    result_precision_score.append(metrics.precision_score(y_test, y_pred))
    result_recall_score.append(metrics.recall_score(y_test, y_pred))
    result_accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
    result_f1_score.append(metrics.f1_score(y_test, y_pred))


#print(len([i for i in range(5, max_depth)]), len(result_precision_score),len(result_recall_score),len(result_accuracy_score),len(result_f1_score))


result_data = {
    "max_depth": [i for i in range(5, max_depth)],
    "precision_score": result_precision_score,
    "recall_score": result_recall_score,
    "accuracy_score": result_accuracy_score,
    "f1_score": result_f1_score
}
df = pd.DataFrame(result_data)
df

,max_depth,precision_score,recall_score,accuracy_score,f1_score
0,5,0.780142,0.245536,0.942281,0.373514
1,6,0.779221,0.267857,0.943376,0.398671
2,7,0.794118,0.301339,0.945565,0.436893
3,8,0.803681,0.292411,0.945409,0.428805
4,9,0.807018,0.308036,0.946348,0.445880
...,...,...,...,...,...
90,95,0.718182,0.529018,0.952448,0.609254
91,96,0.716049,0.517857,0.951822,0.601036
92,97,0.722045,0.504464,0.951666,0.593955
93,98,0.711246,0.522321,0.951666,0.602317


In [56]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig = px.line(df, x="max_depth", y= ["f1_score", "accuracy_score"], title="Lemmatisation results")
fig.show()

[[1, 0.9278339764313276], [2, 0.9278339764313276], [3, 0.9296068411721764], [4, 0.9297111273334029], [5, 0.9311711335905726], [6, 0.9316925643967046], [7, 0.9320054228803838], [8, 0.9322139952028365], [9, 0.931275419751799], [10, 0.9327354260089686], [11, 0.932318281364063], [12, 0.9316925643967046], [13, 0.9298154134946293], [14, 0.9300239858170821], [15, 0.9292939826884973], [16, 0.9285639795599124], [17, 0.9292939826884973], [18, 0.9271039733027427], [19, 0.9248096777557618], [20, 0.9244968192720826], [21, 0.9206382313067056], [22, 0.9203253728230264], [23, 0.9161539263739702], [24, 0.9144853477943476], [25, 0.9147982062780269], [26, 0.911982479924914], [27, 0.9087496089268954], [28, 0.9072896026697257], [29, 0.9055167379288769], [30, 0.9042653039941599], [31, 0.9036395870268016], [32, 0.9040567316717072], [33, 0.9028052977369903], [34, 0.9028052977369903], [35, 0.903222442381896], [36, 0.9017624361247263], [37, 0.900928146834915], [38, 0.900928146834915], [39, 0.9011367191573678], [40, 0.9008238606736886], [41, 0.8987381374491605], [42, 0.8992595682552925], [43, 0.8989467097716133], [44, 0.8973824173532172], [45, 0.8958181249348212], [46, 0.8966524142246324], [47, 0.8961309834185004], [48, 0.8964438419021796], [49, 0.8957138387735948]]

First run
Best result: max detp: 10, 0.9327354260089686 

secound run

Dataset split: 0.9278339764313276

## Upsampeling


## Split data

In [57]:
X_up2 = train_data.drop("label", 1)
y_up2 = train_data.label

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_97800/292283133.py:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [58]:
X_train_up2, X_test_up2, y_train_up2, y_test_up2 = train_test_split(X_up2, y_up2, test_size=0.2, random_state = 17, stratify=y_up2)

In [59]:
print(f"Shape X_train {X_train_up2.shape}")
print(f"Shape X_test {X_test_up2.shape}")

Shape X_train (25569, 14)
Shape X_test (6393, 14)


concat tain and test

In [60]:
df_train_up2 = pd.concat([y_train_up2,X_train_up2], axis=1)
df_test_up2 = pd.concat([y_test_up2,X_test_up2], axis = 1)
df_train_up2.head()

,label,id,tweet,tweet_converted_emojis,n_mentions,hashtags,without_punctuation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
30405,0,30406,when everyone's free when you're in exam mode ...,when everyone's free when you're in exam mode ...,0,"['wtf', 'badtiming', 'guys']",when everyones free when youre in exam mode wt...,when everyones free when youre in exam mode wt...,"['when', 'everyones', 'free', 'when', 'youre',...","['everyones', 'free', 'youre', 'exam', 'mode',...","['wtf', 'badtiming', 'guys']","['everyon', 'free', 'your', 'exam', 'mode', 'w...","['wtf', 'badtim', 'guy']","['everyones', 'free', 'youre', 'exam', 'mode',...","['wtf', 'badtiming', 'guy']"
27807,0,27808,#jacksonville rooster simulation: i want to ...,#jacksonville rooster simulation: i want to ...,0,['jacksonville'],jacksonville rooster simulation i want to cl...,jacksonville rooster simulation i want to cl...,"['jacksonville', 'rooster', 'simulation', 'i',...","['jacksonville', 'rooster', 'simulation', 'wan...",['jacksonville'],"['jacksonvil', 'rooster', 'simul', 'want', 'cl...",['jacksonvil'],"['jacksonville', 'rooster', 'simulation', 'wan...",['jacksonville']
8660,0,8661,@user just run 10kms for @user @user #lovei...,@user just run 10kms for @user @user #lovei...,3,['loveisall'],user just run 10kms for user user loveisall...,user just run 10kms for user user loveisall...,"['user', 'just', 'run', '10kms', 'for', 'user'...","['user', 'run', '10kms', 'user', 'user', 'love...",['loveisall'],"['user', 'run', '10km', 'user', 'user', 'lovei...",['loveisal'],"['user', 'run', '10kms', 'user', 'user', 'love...",['loveisall']
19185,0,19186,@user got the prototype for our new usb today...,@user got the prototype for our new usb today...,1,"['new', 'doncasterphotographer', 'special']",user got the prototype for our new usb today ...,user got the prototype for our new usb today ...,"['user', 'got', 'the', 'prototype', 'for', 'ou...","['user', 'got', 'prototype', 'new', 'usb', 'to...","['new', 'doncasterphotographer', 'special']","['user', 'got', 'prototyp', 'new', 'usb', 'tod...","['new', 'doncasterphotograph', 'special']","['user', 'got', 'prototype', 'new', 'usb', 'to...","['new', 'doncasterphotographer', 'special']"
10355,0,10356,have a &amp; #healthy #fathersday. #runnerda...,have a &amp; #healthy #fathersday. #runnerda...,0,"['healthy', 'fathersday', 'runnerdad', 'eathea...",have a amp healthy fathersday runnerdad eath...,have a amp healthy fathersday runnerdad eath...,"['have', 'a', 'amp', 'healthy', 'fathersday', ...","['amp', 'healthy', 'fathersday', 'runnerdad', ...","['healthy', 'fathersday', 'runnerdad', 'eathea...","['amp', 'healthi', 'fathersday', 'runnerdad', ...","['healthi', 'fathersday', 'runnerdad', 'eathea...","['amp', 'healthy', 'fathersday', 'runnerdad', ...","['healthy', 'fathersday', 'runnerdad', 'eathea..."


Seperate Minortity Class

In [61]:
data_minority = df_train_up2[df_train_up2.label == 1]
data_majority = df_train_up2[df_train_up2.label == 0]
print("length majority", len(data_majority))
print("length minority", len(data_minority))

length majority 23775
length minority 1794


Upsample minority class

In [62]:
data_minority = resample(data_minority, replace = True, n_samples=23775, random_state=55)

Combine the classes

In [63]:
df_tarin_up2 = pd.concat([data_majority, data_minority])
df_tarin_up2.label.value_counts()

0    23775
1    23775
Name: label, dtype: int64

In [64]:
df_tarin_up2.head()

,label,id,tweet,tweet_converted_emojis,n_mentions,hashtags,without_punctuation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
30405,0,30406,when everyone's free when you're in exam mode ...,when everyone's free when you're in exam mode ...,0,"['wtf', 'badtiming', 'guys']",when everyones free when youre in exam mode wt...,when everyones free when youre in exam mode wt...,"['when', 'everyones', 'free', 'when', 'youre',...","['everyones', 'free', 'youre', 'exam', 'mode',...","['wtf', 'badtiming', 'guys']","['everyon', 'free', 'your', 'exam', 'mode', 'w...","['wtf', 'badtim', 'guy']","['everyones', 'free', 'youre', 'exam', 'mode',...","['wtf', 'badtiming', 'guy']"
27807,0,27808,#jacksonville rooster simulation: i want to ...,#jacksonville rooster simulation: i want to ...,0,['jacksonville'],jacksonville rooster simulation i want to cl...,jacksonville rooster simulation i want to cl...,"['jacksonville', 'rooster', 'simulation', 'i',...","['jacksonville', 'rooster', 'simulation', 'wan...",['jacksonville'],"['jacksonvil', 'rooster', 'simul', 'want', 'cl...",['jacksonvil'],"['jacksonville', 'rooster', 'simulation', 'wan...",['jacksonville']
8660,0,8661,@user just run 10kms for @user @user #lovei...,@user just run 10kms for @user @user #lovei...,3,['loveisall'],user just run 10kms for user user loveisall...,user just run 10kms for user user loveisall...,"['user', 'just', 'run', '10kms', 'for', 'user'...","['user', 'run', '10kms', 'user', 'user', 'love...",['loveisall'],"['user', 'run', '10km', 'user', 'user', 'lovei...",['loveisal'],"['user', 'run', '10kms', 'user', 'user', 'love...",['loveisall']
19185,0,19186,@user got the prototype for our new usb today...,@user got the prototype for our new usb today...,1,"['new', 'doncasterphotographer', 'special']",user got the prototype for our new usb today ...,user got the prototype for our new usb today ...,"['user', 'got', 'the', 'prototype', 'for', 'ou...","['user', 'got', 'prototype', 'new', 'usb', 'to...","['new', 'doncasterphotographer', 'special']","['user', 'got', 'prototyp', 'new', 'usb', 'tod...","['new', 'doncasterphotograph', 'special']","['user', 'got', 'prototype', 'new', 'usb', 'to...","['new', 'doncasterphotographer', 'special']"
10355,0,10356,have a &amp; #healthy #fathersday. #runnerda...,have a &amp; #healthy #fathersday. #runnerda...,0,"['healthy', 'fathersday', 'runnerdad', 'eathea...",have a amp healthy fathersday runnerdad eath...,have a amp healthy fathersday runnerdad eath...,"['have', 'a', 'amp', 'healthy', 'fathersday', ...","['amp', 'healthy', 'fathersday', 'runnerdad', ...","['healthy', 'fathersday', 'runnerdad', 'eathea...","['amp', 'healthi', 'fathersday', 'runnerdad', ...","['healthi', 'fathersday', 'runnerdad', 'eathea...","['amp', 'healthy', 'fathersday', 'runnerdad', ...","['healthy', 'fathersday', 'runnerdad', 'eathea..."


In [65]:
X_train_up2 = df_tarin_up2.drop("label", 1)
y_train_up2 = df_tarin_up2.label

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_97800/986097789.py:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [66]:
features = ["lemmatized_tokens", "lemmatized_hashtags", "n_mentions"]

X_train_up2 = X_train_up2[features]
X_test_up2 = X_test_up2[features]

X_train_up2.head()

,lemmatized_tokens,lemmatized_hashtags,n_mentions
30405,"['everyones', 'free', 'youre', 'exam', 'mode',...","['wtf', 'badtiming', 'guy']",0
27807,"['jacksonville', 'rooster', 'simulation', 'wan...",['jacksonville'],0
8660,"['user', 'run', '10kms', 'user', 'user', 'love...",['loveisall'],3
19185,"['user', 'got', 'prototype', 'new', 'usb', 'to...","['new', 'doncasterphotographer', 'special']",1
10355,"['amp', 'healthy', 'fathersday', 'runnerdad', ...","['healthy', 'fathersday', 'runnerdad', 'eathea...",0


### Using TF-IDF Vectorizer

In [67]:
X_train_up2.shape

(47550, 3)

In [68]:
tf = TfidfVectorizer()

X_vec_2 = tf.fit(X_train_up2["lemmatized_tokens"])
X_tarin_up2 = X_vec_2.transform(X_train_up2["lemmatized_tokens"])
X_test_up2 = X_vec_2.transform(X_test_up2["lemmatized_tokens"])

In [69]:
X_tarin_up2.shape

(47550, 35036)

## Fitting model

In [70]:
result_precision_score = []
result_recall_score = []
result_accuracy_score = []
result_f1_score = []
max_depth = 100

for i in range(5, max_depth):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_tarin_up2, y_train_up2)

    y_pred = model.predict(X_test_up2)
    
    result_precision_score.append(metrics.precision_score(y_test_up2, y_pred))
    result_recall_score.append(metrics.recall_score(y_test_up2, y_pred))
    result_accuracy_score.append(metrics.accuracy_score(y_test_up2, y_pred))
    result_f1_score.append(metrics.f1_score(y_test_up2, y_pred))


#print(len([i for i in range(5, max_depth)]), len(result_precision_score),len(result_recall_score),len(result_accuracy_score),len(result_f1_score))


result_data = {
    "max_depth": [i for i in range(5, max_depth)],
    "precision_score": result_precision_score,
    "recall_score": result_recall_score,
    "accuracy_score": result_accuracy_score,
    "f1_score": result_f1_score
}
df_upsampled = pd.DataFrame(result_data)
df_upsampled

,max_depth,precision_score,recall_score,accuracy_score,f1_score
0,5,0.130505,0.542411,0.714688,0.210390
1,6,0.138979,0.571429,0.721883,0.223581
2,7,0.140513,0.611607,0.710621,0.228524
3,8,0.145982,0.640625,0.712185,0.237780
4,9,0.152613,0.658482,0.719850,0.247795
...,...,...,...,...,...
90,95,0.354430,0.625000,0.893947,0.452342
91,96,0.352273,0.622768,0.893321,0.450000
92,97,0.353827,0.629464,0.893477,0.453012
93,98,0.378049,0.622768,0.901768,0.470489


In [71]:
import plotly.express as px


fig = px.line(df_upsampled, x="max_depth", y= ["f1_score", "accuracy_score"], title="Lemmatisation results upsampeled")
fig.show()

# Grid search

In [72]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

## Setup parameters

In [73]:
X.head()

,lemmatized_tokens,lemmatized_hashtags,n_mentions
0,"['user', 'father', 'dysfunctional', 'selfish',...",['run'],1
1,"['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']",2
2,"['bihday', 'majesty']",[],0
3,"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],0
4,"['factsguide', 'society', 'motivation']",['motivation'],0


In [74]:
max_depth = 130
criterion = ['gini', 'entropy']
max_depth = [i for i in range(5, max_depth, 5)]

#min_samples_split
#min_samples_leaf
#min_weight_fraction_leaf
#max_featuresint, float or {“auto”, “sqrt”, “log2”}, default=None


parameters = dict(dec_tree__criterion=criterion,
                  dec_tree__max_depth=max_depth)


## Pipline

In [75]:
dec_tree = DecisionTreeClassifier(random_state=55)

pipe = Pipeline(steps=[('dec_tree', dec_tree)])

## Search

In [76]:
clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X_train, y_train)

print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print(clf_GS.best_estimator_.get_params()['dec_tree'])


Best Criterion: gini
Best max_depth: 55
DecisionTreeClassifier(max_depth=55, random_state=55)
